# Deliverable 2
__Names and Uid__  
Albert Perez Paredes U214020  
Pol Garcia Lopez U214574

### Imports

In [33]:
import nltk
import time
import numpy as np
nltk.download('stopwords')

import sys
from pathlib import Path

[nltk_data] Downloading package stopwords to /home/pol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading the json

In [34]:
document_path = r"../../data/fashion_products_dataset.json"
# sys
# pathlib Path

project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))
import importlib

from myapp.search.load_corpus import load_corpus
docs = load_corpus(document_path)


from myapp.search.algorithms import create_index_tfidf, create_index_tfidf_log, compute_line_docs
from myapp.search.algorithms import search_tfidf
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import seaborn as sns

#Pt2
from myapp.search.algorithms import precision_at_K, select_queries

import myapp.search.algorithms as alg

### Implementation

__Preprocessing documents__

In [35]:
#myapp.search.algorithms create_index_tfidf, compute_line_docs

start_time = time.time()
docs = compute_line_docs(docs)
print("Total time to preprocess documents: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to preprocess documents: 101.34 seconds


# __Part 1__ Indexing

__Index_tfidf__  

In [36]:
start_time = time.time()
num_documents = len(docs)
index, tf, df, idf, title_index = create_index_tfidf(docs, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 109.32 seconds


__Querying__  


In [47]:
# myapp.search.algorithms search tfidf
print("Insert your query (i.e.: women full sleeve sweatshirt cotton):\n")
query = input()
# query = "women full sleeve sweatshirt cotton"
print("\nSearching for: {}\n".format(query)) #Añadido para que en Visual Studio Code se vea la query introducida
ranked_docs = search_tfidf(query, index, idf, tf, title_index)
top = 30

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

Insert your query (i.e.: women full sleeve sweatshirt cotton):


Searching for: women full sleeve sweatshirt cotton


Top 30 results out of 456 for the searched query:

page_id= SWSFMTNHEM5SNHVZ - page_title: Full Sleeve Solid Women Sweatshirt
page_id= SWSFXMFPPZGDQGMW - page_title: Full Sleeve Striped Women Sweatshirt
page_id= SWSFXMFPDVRHYYPH - page_title: Full Sleeve Striped Women Sweatshirt
page_id= SWSFV5JNDCDYTNQY - page_title: Full Sleeve Printed Women Sweatshirt
page_id= SWSFXZWGMGVDUCXA - page_title: Full Sleeve Printed Women Sweatshirt
page_id= SWSFXMFPNZQGZZNE - page_title: Full Sleeve Printed Women Sweatshirt
page_id= SWSFXMFP2RUM86QD - page_title: Full Sleeve Printed Women Sweatshirt
page_id= SWSFV5JNYHKCWHHX - page_title: Full Sleeve Color Block Women Sweatshirt
page_id= SWSFXYRYTHZWSZPE - page_title: Full Sleeve Graphic Print Women Sweatshirt
page_id= SWSFWTND3EPMFCQD - page_title: Full Sleeve Solid Women Sweatshirt
page_id= SWSFWTNDJFCF72WU - page_title: Full Sleeve Sol

# __Part 2__ Evaluation

In [38]:
queries_real = pd.read_csv(r"../../data/validation_labels.csv")
queries_real.head(40) #solo hay 40 de momento

,title,pid,query_id,labels
0,Full Sleeve Printed Women Sweatshirt,SWSFFVKBCQG5FHPF,1,1
1,Full Sleeve Striped Women Sweatshirt,SWSFJY5ZFHQ7HXKW,1,0
2,Full Sleeve Printed Women Sweatshirt,SWSFUY89NHMZHZPX,1,1
3,Full Sleeve Graphic Print Women Sweatshirt,SWSFXQ5YX6RZKHP4,1,1
4,Full Sleeve Solid Women Sweatshirt,JCKFTZBC3DMCVYXH,1,0
5,Full Sleeve Graphic Print Women Sweatshirt,SWSFWCTDHRABJFCR,1,1
6,Typography Women V Neck Multicolor T-Shirt (P...,TSHFUH9FABXHSDPJ,1,0
7,Full Sleeve Printed Women Sweatshirt,SWSFVGKENHBSVUWJ,1,1
8,Full Sleeve Solid Women Sweatshirt,JCKF7M8DNBB6WZ8Y,1,1
9,Full Sleeve Graphic Print Women Sweatshirt,SWSFWCSPEGMDH8FV,1,1


__Pt2.2__

This was a try to implement logarithmic scale for tf, but fo now it gave us the same results as not using it so we won't use it

In [ ]:
# importlib.reload(alg)
# from myapp.search.algorithms import create_index_tfidf_log

# start_time = time.time()
# num_documents = len(docs)
# index, tf, df, idf, title_index = create_index_tfidf(docs, num_documents)
# index_log, tf_log, df, idf, title_index = create_index_tfidf_log(docs, num_documents)
# print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

pre = queries_real[queries_real["query_id"] == 1]
pre.head()
queries = ["women full sleeve sweatshirt cotton", "men slim jeans blue"]
ranked_docs_t = search_tfidf(queries[0], index, idf, tf, title_index)

print(ranked_docs_t[:10])

for d_id in ranked_docs_t:
    for q_id in pre["pid"]:
        if d_id == q_id:
            print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

,title,pid,query_id,labels
0,Full Sleeve Printed Women Sweatshirt,SWSFFVKBCQG5FHPF,1,1
1,Full Sleeve Striped Women Sweatshirt,SWSFJY5ZFHQ7HXKW,1,0
2,Full Sleeve Printed Women Sweatshirt,SWSFUY89NHMZHZPX,1,1
3,Full Sleeve Graphic Print Women Sweatshirt,SWSFXQ5YX6RZKHP4,1,1
4,Full Sleeve Solid Women Sweatshirt,JCKFTZBC3DMCVYXH,1,0


Precision

In [ ]:
from myapp.search.algorithms import precision_at_K

k = 5000
# queries = ["Full Sleeve Printed Women Sweatshirt"]
# queries_sel = select_queries(queries, queries_real)

# for i, q in enumerate(queries_sel):
#     ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
#     precision_t = precision_at_K(q, ranked_docs_t, k)
#     print("The query {%s} has precision %0.3f at k=%d" % (queries[i], precision_t, k))

queries = ["women full sleeve sweatshirt cotton",
           "men slim jeans blue"
          ] # exercice 2.2 queries

queries_id = list(range(1,1+len(queries))) # [0, 1, ..., n]
queries_sel = select_queries(queries, queries_real) # list of dataframe of queries

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    precision_t = precision_at_K(q, ranked_docs_t, k)
    print("%0.3f" % precision_t)

0.0000000000000
0.0000000000000


Recall

In [49]:
from myapp.search.algorithms import recall_at_K

k = 20

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    recall_t = recall_at_K(q, ranked_docs_t, k)
    print("%0.3f" % recall_t)

0.000
0.000


Average Precision

In [50]:
from myapp.search.algorithms import average_precision

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    avgprecision_t = average_precision(q, ranked_docs_t)
    print("%0.3f" % avgprecision_t)

0.000
0.000


F1_score

In [51]:
from myapp.search.algorithms import f1_score_at_K, f1_score_at_K_optimized

k = 20

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    # f1_score = f1_score_at_K(q, ranked_docs_t, k)
    f1_score_opt = f1_score_at_K_optimized(q, ranked_docs_t, k)
    print("%0.3f" % f1_score_opt)

0.000
0.000


MAP

In [52]:
from myapp.search.algorithms import mean_average_precision

queries_only = queries

map_map = mean_average_precision(queries_real, index, idf, tf, title_index)
print("%0.3f" % map_map)

0.094


MRR

In [76]:
importlib.reload(alg)
from myapp.search.algorithms import mean_reciprocal_rank

k = 2000
print(queries)
mrr = mean_reciprocal_rank(queries_real, queries, index, idf, tf, title_index, k)
print("%0.3f" % mrr)

['women full sleeve sweatshirt cotton', 'men slim jeans blue']
[1, 2]
not_found
not_found
0.000


NDCG

In [54]:
importlib.reload(alg)
from myapp.search.algorithms import NDCG

k = 20

for i, q in enumerate(queries_sel):
    ranked_docs_t = search_tfidf(queries[i], index, idf, tf, title_index)
    ndcg = NDCG(q, ranked_docs_t, k)
    print("%0.3f" % ndcg)

0.000
0.000
